In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as sco
import matplotlib.pyplot as plt
import random

In [ ]:
def zjh(arr):
    arr_t=arr.transpose()

    #计算重叠矩阵
    overlap_matrix=np.dot(arr_t,arr)
    overlap_matrix=(overlap_matrix.T+overlap_matrix)/2
    #计算特征值、特征向量
    eigvalue,eigvector=np.linalg.eig(overlap_matrix)

    #处理对角矩阵
    eigvalue=eigvalue**(-0.5)
    DiagMat=np.diag(eigvalue)

    #计算最终正交矩阵
    tmp=np.dot(eigvector,DiagMat)
    TSMat=np.dot(tmp,eigvector.transpose())
    OrthMat=np.dot(arr,TSMat)
    
    return OrthMat

def data_handle(mat):
    for i in range(mat.shape[1]):
        arr = np.array(mat[:,i])
        M = np.median(arr)
        MAD = 1.483 * np.median(np.abs(arr - M))
        arr[arr > M + 3 * MAD] = M + 3 * MAD
        arr[arr < M - 3 * MAD] = M - 3 * MAD
        mat[:,i] = arr
            
    for i in range(mat.shape[1]):
        arr = np.array(mat[:,i])
        weight = np.array(len(arr) * [1 / len(arr)])
        weighted_avarage = np.dot(weight.T, arr)
        arr = (arr - weighted_avarage) / arr.std()
        mat[:,i] = arr
    return mat

In [ ]:
#数据处理
change_data_1=pd.read_csv("change.csv",dtype={"stock_code":str})
change_data_2=pd.read_csv("change_2.csv",dtype={"stock_code":str})
change_data=pd.concat([change_data_1,change_data_2])

change_data.rename(columns={"stock_code":"code","trade_date":"datetime"},inplace=True)
change_data["datetime"]=change_data["datetime"].apply(lambda x:x[0:10])
change_data["code"]=change_data["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")

change_data.sort_values(by=["datetime","code"],ascending=[True,True],inplace=True)

change_data.set_index("code",inplace=True)

grouped=change_data.groupby("datetime")
date_list=list(change_data["datetime"].drop_duplicates())
code_list=list(change_data.index.drop_duplicates())
join=set(code_list)
union=set(code_list)
for date in date_list:
    df=grouped.get_group(date)
    stock_set=set(df.index)
    join=stock_set&join
    union=stock_set|union
other_list=list(union-join)
change_data.drop(other_list,inplace=True)

drop_list=random.sample(list(join),1135)
change_data.drop(drop_list,inplace=True)

change_data.reset_index(inplace=True)


data=pd.read_csv("data_needs.csv")
date=list(data["datetime"].drop_duplicates())[-1]
df=data.groupby("datetime").get_group(date)
dic=dict(zip(df["code"],df["industry_code"]))
change_data["industry_code"]=change_data["code"].apply(lambda x:dic.get(x,0))

drop_list=list(set(change_data["code"][change_data["industry_code"]==0]))
change_data.set_index("code",inplace=True)
change_data.drop(drop_list,inplace=True)
change_data.reset_index(inplace=True)

stock_ret_df=change_data.pivot_table("change_rate",index="datetime",columns="code")

In [ ]:
alpha_df=pd.read_csv("growth_factor.csv")
del alpha_df["industry_code"]

code_list=list(alpha_df["code"].drop_duplicates())
alpha_df.set_index("code",inplace=True)

drop_code=set(code_list)-set(stock_ret_df.columns)
alpha_df.drop(drop_code,inplace=True)

alpha_df.reset_index()

alpha_df.sort_values(by=["datetime","code"],ascending=[True,True],inplace=True)

alpha_df=pd.merge(alpha_df,change_data,on=["datetime","code"])
alpha_df.fillna(0,inplace=True)

In [ ]:
mcap_data_1=pd.read_csv("mcap.csv",dtype={"stock_code":str})
mcap_data_2=pd.read_csv("mcap_2.csv",dtype={"stock_code":str})
mcap_data=pd.concat([mcap_data_1,mcap_data_2])

mcap_data.rename(columns={"stock_code":"code","trade_date":"datetime"},inplace=True)
mcap_data["datetime"]=mcap_data["datetime"].apply(lambda x:x[0:10])
mcap_data["code"]=mcap_data["code"].apply(lambda x:x+".SH" if x[0]=="6" else x+".SZ")

new_alpha_df=pd.merge(alpha_df,mcap_data,on=["datetime","code"])

In [ ]:
ret_df=stock_ret_df

for code in ret_df.columns:
    ret_df[code+"_In_change"]=np.array(np.log(1+ret_df[code]/100))[::-1]
    ret_df[code+"_21d_ret"]=(np.array(np.exp(ret_df[code+"_In_change"].rolling(21).apply(np.sum)))[::-1]-1)*100
    ret_df[code+"_21d_ret"]=ret_df[code+"_21d_ret"].shift(-1)
    del ret_df[code+"_In_change"]

stock_num=len(stock_ret_df.columns)
df=pd.DataFrame(stock_ret_df.iloc[:,int(stock_num/2):stock_num]).stack()

df=df.reset_index()
df["code"]=df["code"].apply(lambda x:x[0:9])
new_alpha_df=pd.merge(new_alpha_df,df,on=["datetime","code"])
new_alpha_df.rename(columns={0:"21d_ret"},inplace=True)

In [ ]:
zz500=pd.read_csv("zz500.csv")
zz500["In_change"]=np.array(np.log(1+zz500["change_rate"]/100))[::-1]
zz500["21d_ret"]=(np.array(np.exp(zz500["In_change"].rolling(21).apply(np.sum)))[::-1]-1)*100
zz500["21d_ret"]=zz500["21d_ret"].shift(-1)


zz500.rename(columns={"21d_ret":"mkt_ret","trade_date":"datetime"},inplace=True)
zz500["datetime"]=zz500["datetime"].apply(lambda x:x[0:10])
del zz500["index_code"]
del zz500["In_change"]
del zz500["change_rate"]
new_alpha_df=pd.merge(new_alpha_df,zz500,on="datetime")

In [ ]:
new_alpha_df

In [ ]:
date_list=list(new_alpha_df["datetime"].drop_duplicates())
code_list=list(new_alpha_df["code"].drop_duplicates())

In [ ]:
date_list=date_list[:-21]
factor_list=["EGRO","EGIB","EPIBS","SGRO"]
factor_ret_df=pd.DataFrame(columns=factor_list,index=date_list)

In [ ]:
alpha_df=new_alpha_df
grouped=alpha_df.groupby("datetime")

for date in date_list:
    df=grouped.get_group(date)

    mktv=np.array(df["mcap"])
    mktv_pct=np.sqrt(mktv)/np.sum(np.sqrt(mktv))
    weight=mktv_pct**(0.5)
    
    y=np.array(df["21d_ret"]-df["mkt_ret"])
    y=np.dot(np.diag(weight),y)
    
    indus_factor=np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
    style_factor=np.array(df[factor_list])
    style_factor=data_handle(zjh(data_handle(style_factor)))
    x=np.column_stack([style_factor,indus_factor])
    x=np.dot(np.diag(weight),x)
    
    model=sm.OLS(y,x)
    result=model.fit()
    factor_ret=result.params[0:4]
    factor_ret_df.loc[date]=factor_ret

In [ ]:
factor_ret_df=pd.concat([factor_ret_df,pd.DataFrame(columns=range(12),index=date_list)],axis=1)

In [ ]:
for i in range(251,len(date_list)-21):
    window=range(i-251,i+1)
    sub_df=factor_ret_df.iloc[window,0:4]
    factor_ret_mean=np.array(sub_df.apply(np.mean,axis=0))
    factor_ret_df.iloc[i+21,4:8]=np.abs(factor_ret_mean)/np.sum(np.abs(factor_ret_mean))
    
    factor_ret_std=np.array(sub_df.apply(np.std,axis=0))
    factor_ret_df.iloc[i+21,8:12]=np.abs(factor_ret_std**-1)/np.sum(np.abs(factor_ret_std**-1))
    
    factor_ret_df.iloc[i+21,12:16]=np.array([0.18,0.11,0.24,0.47])


In [ ]:
R_square_df=pd.DataFrame(index=date_list[272:len(date_list)-21],columns=["mean_weight","var_weight","fixed_weight"])

In [ ]:
for date in date_list[272:len(date_list)-21]:
    print(date,end=" ")
    df=grouped.get_group(date)

    mktv=np.array(df["mcap"])
    mktv_pct=np.sqrt(mktv)/np.sum(np.sqrt(mktv))
    weight=mktv_pct**(0.5)
    
    y=np.array(df["21d_ret"]-df["mkt_ret"])
    y=np.dot(np.diag(weight),y)
    
    indus_factor=np.array(sm.categorical(np.array(df["industry_code"])))[:, 1:]
    
    
    #因子收益率均值作为因子权重
    style_factor=np.array(df[factor_list])
    factor_weight=np.array(factor_ret_df.ix[date,4:8])
    new_factor=np.dot(style_factor,factor_weight.T)
    new_factor=data_handle(new_factor.reshape(len(new_factor),1))
    x=np.column_stack([new_factor,indus_factor])
    x=np.dot(np.diag(weight),x)
    x=np.array(x,dtype=float)
    
    model=sm.OLS(y,x)
    result=model.fit()
    R_square=result.rsquared_adj
    R_square_df.loc[date,"mean_weight"]=R_square
    
    
    #因子收益率波动率倒数作为因子权重
    style_factor=np.array(df[factor_list])
    factor_weight=np.array(factor_ret_df.ix[date,8:12])
    new_factor=np.dot(style_factor,factor_weight.T)
    new_factor=data_handle(new_factor.reshape(len(new_factor),1))
    x=np.column_stack([new_factor,indus_factor])
    x=np.dot(np.diag(weight),x)
    x=np.array(x,dtype=float)
    
    model=sm.OLS(y,x)
    result=model.fit()
    R_square=result.rsquared_adj
    R_square_df.loc[date,"var_weight"]=R_square
    
    
    #固定权重
    style_factor=np.array(df[factor_list])
    factor_weight=np.array(factor_ret_df.ix[date,12:16])
    new_factor=np.dot(style_factor,factor_weight.T)
    new_factor=data_handle(new_factor.reshape(len(new_factor),1))
    x=np.column_stack([new_factor,indus_factor])
    x=np.dot(np.diag(weight),x)
    x=np.array(x,dtype=float)
    
    model=sm.OLS(y,x)
    result=model.fit()
    R_square=result.rsquared_adj
    R_square_df.loc[date,"fixed_weight"]=R_square

    

In [ ]:
R_square_df.mean(axis=0)

In [ ]:
R_square_df.index=pd.to_datetime(R_square_df.index)

In [ ]:
ax = R_square_df.plot() 
fig = ax.get_figure()
fig.savefig('R_square.jpg',dpi=500)